Handle Evolving Workflows
=========================

For some workflows we don't know the extent of the computation at the outset.  We need to do some computation in order to figure out the rest of the computation that we need to do.  The computation grows and evolves as we do more work.

As an example, consider a situation where you need to read many files and then based on the contents of those files, fire off additional work.  You would like to read the files in parallel, and then within each file expose more parallelism.

This example goes through three ways to handle this situation using [Dask Futures](https://docs.dask.org/en/latest/futures.html)

1.  Using `as_completed`
2.  Using `async/await`
3.  Launching tasks from tasks

But first, lets run our code sequentially.

0: Sequential code
------------------

In [1]:
filenames = ["file.{}.txt".format(i) for i in range(10)]

filenames[:3]

['file.0.txt', 'file.1.txt', 'file.2.txt']

In [2]:
import random, time


def parse_file(fn: str) -> list:
    """ Returns a list work items of unknown length """
    time.sleep(random.random())
    return [random.random() for _ in range(random.randint(1, 10))]

def process_item(x: float):
    """ Process each work item """
    time.sleep(random.random() / 4)
    return x + 1

In [3]:
%%time

# This takes around 10-20s

results = []

for fn in filenames:
    L = parse_file(fn)
    for x in L:
        out = process_item(x)
        results.append(out)

CPU times: user 1.14 ms, sys: 3.38 ms, total: 4.52 ms
Wall time: 11.1 s


Start Dask Client
-----------------

We'll need a Dask client in order to manage dynamic workloads

In [4]:
from dask.distributed import Client

client = Client(processes=False, n_workers=1, threads_per_worker=6)
client

Client Scheduler: inproc://10.20.0.5/10326/1 Dashboard: http://10.20.0.5:8787/status,Cluster Workers: 1 Cores: 6 Memory: 8.36 GB


1: Use as_completed
-------------------

The [as_completed](https://docs.dask.org/en/latest/futures.html#distributed.as_completed) iterator lets us handle futures as they complete.  We can then submit more data on the fly.

-   We submit a task for each of our filenames
-   We also compute the length of each of the returned lists
-   As those lengths return, we submit off a new task to get each item of that list.  We do this at higher priority, so that we process existing data before we collect new data.
-   We wait on all of the returned results

In [5]:
%%time

from dask.distributed import as_completed
import operator

lists = client.map(parse_file, filenames, pure=False)
lengths = client.map(len, lists)

mapping = dict(zip(lengths, lists))

futures = []

for future in as_completed(lengths):
    n = future.result()
    L = mapping[future]
    for i in range(n):
        new = client.submit(operator.getitem, L, i, priority=1)
        new = client.submit(process_item, new, priority=1)
        futures.append(new)
        
client.gather(futures)

CPU times: user 226 ms, sys: 31 ms, total: 257 ms
Wall time: 2.05 s


[1.4889764819170694,
 1.7040972248352,
 1.1571474201463028,
 1.5454059923738144,
 1.6980484122985602,
 1.5859244291587444,
 1.6310193600027705,
 1.3543871599536126,
 1.7090320457207968,
 1.782310480103702,
 1.2318069548963009,
 1.924098536790956,
 1.7257300973393783,
 1.756806464424603,
 1.9067889891412517,
 1.7960690730541036,
 1.1359838000358866,
 1.494203578173991,
 1.6488603571337328,
 1.096526036768188,
 1.667648240408289,
 1.4061562556267293,
 1.8326369431493768,
 1.689991831494695,
 1.461987008316703,
 1.5572633975292502,
 1.1939021069804852,
 1.8616217271346318,
 1.3095632374113033,
 1.9230506916990584,
 1.481340969439661,
 1.1779449085085194,
 1.8649200288661523,
 1.3464213388856434,
 1.1238484098384038,
 1.586864424747724,
 1.7452039000794763,
 1.336652841431686,
 1.4540782870643398,
 1.0726687760516622,
 1.1712907905220624,
 1.5879342455066605,
 1.2118994236242568,
 1.413307441129111,
 1.847219575515521,
 1.517026028340272,
 1.841083374567698,
 1.051453715707106,
 1.71190316

2: Use async/await to handle single file processing locally
-----------------------------------------------------------

We can also handle the concurrency here within our local process.  This requires you to understand async/await syntax, but is generally powerful and arguably simpler than the `as_completed` approach above.

In [6]:
import asyncio

async def f(fn):
    """ Handle the lifecycle of a single file """
    future = client.submit(parse_file, fn, pure=False)
    length_future = client.submit(len, future)
    length = await length_future
    
    futures = [client.submit(operator.getitem, future, i, priority=10) 
               for i in range(length)]
    futures = client.map(process_item, futures, priority=10)
    return futures

async def run_all(filenames):
    list_of_list_of_futures = await asyncio.gather(*[f(fn) for fn in filenames])
    futures = sum(list_of_list_of_futures, [])
    return await client.gather(futures)


We now need to run this function in the same event loop as our client is running.  If we had started our client asynchronously, then we could have done this:

```python
client = await Client(asynchronous=True)

await run_all(filenames)
```

However, because we started our client without the `asynchronous=True` flag the event loop is actually running in a separate thread, so we'll have to ask the client to run this for us.

In [7]:
client.sync(run_all, filenames)

[1.4955011638438545,
 1.7469930673817191,
 1.6975987015605898,
 1.0147919960034988,
 1.6277505556265015,
 1.2173602374254293,
 1.0837302417191976,
 1.075590249167107,
 1.751379585957811,
 1.1558119833323104,
 1.5951921813750392,
 1.5938748947458778,
 1.3521045355361032,
 1.486320307441011,
 1.4071211652659725,
 1.8962867765615183,
 1.7013899662840555,
 1.581929393957752,
 1.743438483469316,
 1.588365631608937,
 1.4357359794819144,
 1.6100986333769443,
 1.8607112524725453,
 1.169627372096319,
 1.4986908661246172,
 1.2045314363348538,
 1.498524731081742,
 1.8788955151997093,
 1.1331117100806436,
 1.8921566636861296,
 1.5100141950145791,
 1.2366869166893286,
 1.4856696026896308,
 1.7026049480958463,
 1.131735959036471,
 1.9541865440830333,
 1.2389767534512655,
 1.3448520965504818,
 1.5367453215965163,
 1.885968667011714,
 1.1367656612942634,
 1.8301705622711437,
 1.9165346105505487,
 1.2891887330068843,
 1.292268723709069,
 1.0843511918961752,
 1.5361764003193321,
 1.3989506356835992,
 1.

3: Submit tasks from tasks
--------------------------

We can also submit tasks that themselves submit more tasks.  See [documentation here](https://docs.dask.org/en/latest/futures.html#submit-tasks-from-tasks).

In [8]:
%%time

from dask.distributed import get_client, secede, rejoin

def f(fn):
    L = parse_file(fn)
    client = get_client()
    
    futures = client.map(process_item, L, priority=10)
    secede()
    results = client.gather(futures)
    rejoin()
    return results

futures = client.map(f, filenames, pure=False)
results = client.gather(futures)

CPU times: user 160 ms, sys: 11.3 ms, total: 171 ms
Wall time: 1.77 s
